# Aggregating and Sorting Data

## Metadata

- Teaching: 60
- Exercises: 0

## Questions

- How do we calculate summary statistics?
- How do we group data?
- How do null values affect calculations?

## Objectives

- Introduce aggregation calculations in pandas
- Introduce grouping in pandas
- Learn about how pandas handles null values

As always, we'll begin by importing pandas and reading our CSV:

In [ ]:
import pandas as pd

surveys = pd.read_csv("data/surveys.csv")

## Aggregating data

Aggregation allows us to describe the data in our dataframe by calculating totals (like the number of records) and statistics (like the mean value of a column). pandas allows us to run these calculations on dataframes or subsets.

Suppose we need to know how many records are in our dataset. We've already seen that we can use the `info()` method to get high-level about the dataset, including the number of entries. What if just wanted the number of rows? One approach is to use the built-in function `len()`, which is used to calculate the number of items in an object (for example, the number of characters in a string or the number of items in a list). When used on a dataframe, `len()` returns the number of rows:

In [ ]:
len(surveys)

pandas provides a suite of aggregation methods that go beyond this simple case. For example, we can count the number of non-NaN values in each column using `count()`:

In [ ]:
surveys.count()

Or we can find out the total weight of all individuals in grams using `sum()`:

In [ ]:
surveys["weight"].sum()

Other aggregation methods supported by pandas include `min()`, `max()`, and `mean()`. These methods all ignore NaNs, so missing data does not affect these calculations. 

## Challenge

Calculate the total weight, average weight, minimum and maximum weights for all animals caught over the duration of the survey. Can you modify your code so that it outputs these values only for weights between 5 and 10 grams?

In [ ]:
# Create a subset of only the animals between 5 and 10 grams
weights = surveys[(surveys["weight"] > 5) & (surveys["weight"] < 10)]["weight"]

# Display aggregation calculations using a dict
{
    "sum": weights.sum(),
    "mean": weights.mean(),
    "min": weights.min(),
    "max": weights.max(),
}

To quickly generate summary statistics, we can use the `describe()` method instead of calling each aggregation method separately. When we use this method on a dataframe, it calculates stats for all columns with numeric data:

In [ ]:
surveys.describe()

You can see that `describe()` isn't picky: It includes both ID and date columns in its results. Notice also that counts differ in between columns. This is because `count()` only counts non-NaN rows.

If desired, we can also describe a single column at a time:

In [ ]:
surveys["weight"].describe()

If we need more control over the output (for example, if we want to calculate the total weight of all animals, as in the challenge above), pandas provides the `agg()` method, which allows us to specify methods by column. The argument passed to this method is a `dict`. Each key must be a column name and each value a list of the names of aggregation methods. To calculate the total weight, mean weight, and mean hindfoot length of all records in the survey, we can use:

In [ ]:
surveys.agg({"weight": ["sum", "mean"], "hindfoot_length": ["mean"]})

## Grouping data

Now, let's find out how many individuals were counted for each species. We do this using `groupby()`, which creates an object similar to a dataframe where rows are grouped by the data in one or more columns. To group by species_id, use:

In [ ]:
grouped = surveys.groupby("species_id")

When we aggregate grouped data, pandas makes separate calculations for each member of the group. In the example below, we'll calculate the number of times each species appears in the dataset. Rather than outputting the full dataframe, we'll limit the count to a single column. Because count ignores NaN cells, it's good practice to use a column that does not contain nulls. Record ID fields are a often a good choice, but any field that is populated for every row will work.

In [ ]:
grouped["species_id"].count()

To group by multiple columns, we can pass a list to `groupby()`:

In [ ]:
surveys.groupby(["species_id", "year"])["record_id"].count()

## Challenge

Write statements to answer the following questions:

1. How many individuals were counted in each year in total?
2. How many were counted each year, for each different species?
3. What was the average weight of each species in each year?
4. How many individuals were counted for each species that was observed more than 10 times?

Can you get the answer to both 2 and 3 in a single query?

### Show me the solution to challenge 1

How many individuals were counted in each year in total?

In [ ]:
# Individual counts per year
surveys.groupby("year")["record_id"].count()

### Show me the solution to challenge 2

How many individuals were counted each year for each different species?

In [ ]:
# Individual counts by species and year
result = surveys.groupby(["year", "species_id"])["record_id"].count()

# Use the option_context context manager to show all rows
with pd.option_context("display.max_rows", None):
    print(result)

### Show me the solution to challenge 3

What was the average weight of each species in each year?

In [ ]:
# Mean weight by species and year
result = surveys.groupby(["year", "species_id"])["weight"].mean()

# Use the option_context context manager to show all rows
with pd.option_context("display.max_rows", None):
    print(result)

### Show me the solution to challenge 4

How many individuals were counted for each species that was observed more than 10 times?

In [ ]:
# Counts by species that appear more than 10 times
species = surveys.groupby("species_id")["record_id"].count()
species[species > 10]

### Show me the solution to challenges 2 and 3 combined

Calculate the number of individuals observed and the average weight for each species in each year.

Note that weight contains NaNs, so counts for record_id and weight differ.

In [ ]:
# Counts and mean weight by species and year
surveys.groupby(["year", "species_id"]).agg(
    {"record_id": "count", "weight": ["count", "mean"]}
)

## Handling missing data

As we've discussed, some columns in the surveys dataframe have the value NaN instead of text or numbers. NaN, short for "not a number," is a special type of float used by pandas to represent missing data. When reading from a CSV, as we have done throughout this lesson, pandas interprets certains values as NaN. NaNs are excluded from groups and most aggregation calculations in pandas, including counts.

## Defaults

See *na_values* in the [pd.read_csv() documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) for the values that pandas interprets as NaN by default.

When analyzing a dataset, it is critical to understand how missing data is represented. Failing to do so may introduce errors into the analysis. The ecology dataset used in this lesson uses empty cells to represent missing data, but other disciplines have different conventions. For example, some geographic datasets use -9999 to represent null values. Failure to convert such values to NaN will result in significant errors on any calculations performed on that dataset.

In some cases, it can be useful to fill in cells containing NaN with a non-null value. For example, the `groupby()` method excludes NaN cells. When looking at sex, the following code counts only those cells with either F (female) or M (male):

In [ ]:
surveys.groupby("sex")["record_id"].count()

But not all records specify a sex. To include records where sex was not specified, we can use the `fillna()` method on the sex column. This method replaces each NaN with the first argument passed to the function call. To replace all NaN values in sex with "U", use:

In [ ]:
surveys["sex"] = surveys["sex"].fillna("U")

The calculation now includes all records in the dataframe:

In [ ]:
surveys.groupby("sex")["record_id"].count()

In other cases, we may want to ignore rows that contain NaNs. This can be done using `dropna()`:

In [ ]:
surveys = surveys.dropna()

This method returns a copy of the dataframe containing only those rows that have valid data in every field.

## Visualizing groups of data

Grouping data is a common operation when visualizing data.

In [ ]:
import plotly.express as px

fig = px.scatter(surveys, x="weight", y="hindfoot_length", color="species_id")
fig.show()

In [ ]:
surveys = surveys.sort_values("species_id")
fig = px.scatter(surveys, x="weight", y="hindfoot_length", color="species_id")
fig.show()

In [ ]:
fig = px.box(surveys, x="species_id", y="weight", color="species_id")
fig.show()

## Keypoints

- Calculate individual summary statistics using dataframe methods like `mean()`, `max()`, and `min()`
- Calculate multiple summary statistics at once using the dataframe methods `describe()` and `agg()`
- Group data by one or more columns using the `groupby()` method
- pandas uses NaN to represent missing data in a dataframe
- Failing to consider how missing data is interpreted in a dataset can introduce errors into calculations